In [48]:
import os
from dotenv import load_dotenv
from minio import Minio
import duckdb
import tempfile

In [49]:
# Carrega variáveis de ambiente
load_dotenv()

True

In [50]:
# Configurações (via .env)
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")
MINIO_ACCESS = os.getenv("MINIO_ACCESS_KEY")
MINIO_SECRET = os.getenv("MINIO_SECRET_KEY")
BUCKET_NAME = os.getenv("MINIO_BUCKET")

In [51]:
# Connect to Minio
minio_client = Minio(
    MINIO_ENDPOINT,
    access_key= MINIO_ACCESS,
    secret_key= MINIO_SECRET,
    secure= False                   # True se estiver usando HTTPS
    
)

In [52]:
# Minimal connection test
try:
    minio_client.list_buckets()  # Simplest API call to test connectivity
    print("MinIO connection working")
except Exception as e:
    print(f"Connection failed: {str(e)}")

MinIO connection working


In [53]:
# Nomes dos buckets
landing_bucket = "test-bucket"               #"landing-zone-integrated"  # Bucket de origem
structured_bucket = "raw-zone-bronze"        # Bucket de destino

In [54]:
input_file = "campeonato_brasileiro_2024.parquet"

In [55]:
# Criar conexao com duckdb em memoria
conn = duckdb.connect(':memory')

In [56]:
# Download do arquivo
tempfile.NamedTemporaryFile(suffix='.parquet')
minio_client.fget_object(landing_bucket, input_file, input_file)

In [57]:
# Carregar e transformar dados
resultado = conn.execute(f"""
            SELECT pos, 
                   Equipevde,
                   Pts
                   FROM read_parquet('{input_file}')
            """).df()

In [58]:
display(resultado)

,Pos,Equipevde,Pts
0,1,Botafogo (C),79
1,2,Palmeiras,73
2,3,Flamengo,70
3,4,Fortaleza,68
4,5,Internacional,65
5,6,São Paulo,59
6,7,Corinthians,56
7,8,Bahia,53
8,9,Cruzeiro,52
9,10,Vasco da Gama,50


In [59]:
# 1. Criar arquivo temporário
temp_file = tempfile.NamedTemporaryFile(suffix='.parquet', delete=False)
resultado.to_parquet(temp_file.name)
temp_file.close()  # Fechar o arquivo para liberar para upload

# 2. Fazer upload para o MinIO
minio_client.fput_object(
    "raw-zone-bronze",
    "brasileiro_processado.parquet",
    temp_file.name
)

# 3. Limpar o arquivo temporário
os.unlink(temp_file.name)

print("Arquivo salvo com sucesso em raw-zone-bronze/brasileiro_processado.parquet")

Arquivo salvo com sucesso em raw-zone-bronze/brasileiro_processado.parquet
